In [2]:
import tensorflow as tf
import numpy as np
import json
import datetime
import os

In [3]:
# Path file tracker_memory.json
TRACKER_FILE_PATH = "result/tracker_memory.json"

In [4]:
# Data Dummy untuk train
data = np.array([
    [170, 70, 25, 1],  # tinggi, berat, usia, gender (1: male, 0: female)
    [160, 55, 30, 0],
    [180, 80, 22, 1],
    [165, 60, 40, 0],
    [175, 75, 35, 1]
])

labels = np.array([
    [2000, 112],  # kalori, protein
    [1800, 88],
    [2500, 128],
    [1500, 72],
    [2200, 120]
])


In [5]:
# Membuat Model TensorFlow
def build_tracker_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu", input_shape=(4,)),  # Input: height, weight, age, gender
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(2, activation="linear")  # Output: calories, protein
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Latih Model
model = build_tracker_model()
model.fit(data, labels, epochs=100, verbose=0)

# Simpan Model ke File
model.save("tracker_model.h5")

In [9]:
# Fungsi untuk Prediksi
def predict_daily_targets(height_cm, weight_kg, age, gender):
    model = load_tracker_model()
    input_data = np.array([[height_cm, weight_kg, age, 1 if gender.lower() == "male" else 0]])
    prediction = model.predict(input_data)
    return {"calories": int(prediction[0][0]), "protein": int(prediction[0][1])}

# Fungsi untuk Menyimpan Data User
def add_food_entry(food_name, calories, protein, fat, carbs, user_id=1):
    entry = {
        "food_name": food_name,
        "calories": calories,
        "protein": protein,
        "fat": fat,
        "carbohydrate": carbs,
        "timestamp": datetime.datetime.now().isoformat(),
        "user_id": user_id
    }
    # Load existing tracker memory
    if os.path.exists(TRACKER_FILE_PATH):
        with open(TRACKER_FILE_PATH, "r") as f:
            tracker_memory = json.load(f)
    else:
        tracker_memory = {}

    # Tambahkan entri berdasarkan tanggal
    date_key = datetime.datetime.now().strftime("%Y-%m-%d")
    if date_key not in tracker_memory:
        tracker_memory[date_key] = []
    tracker_memory[date_key].append(entry)

    # Simpan kembali ke file
    with open(TRACKER_FILE_PATH, "w") as f:
        json.dump(tracker_memory, f, indent=4)

    return entry

In [11]:
# Fungsi untuk Mendapatkan Total Nutrisi Harian
def get_daily_totals(user_id=1):
    date_key = datetime.datetime.now().strftime("%Y-%m-%d")

    if os.path.exists(TRACKER_FILE_PATH):
        with open(TRACKER_FILE_PATH, "r") as f:
            tracker_memory = json.load(f)
    else:
        return {"calories": 0, "protein": 0, "fat": 0, "carbohydrate": 0}

    if date_key not in tracker_memory:
        return {"calories": 0, "protein": 0, "fat": 0, "carbohydrate": 0}

    totals = {"calories": 0, "protein": 0, "fat": 0, "carbohydrate": 0}
    for entry in tracker_memory[date_key]:
        if entry.get("user_id") == user_id:
            totals["calories"] += entry["calories"]
            totals["protein"] += entry["protein"]
            totals["fat"] += entry["fat"]
            totals["carbohydrate"] += entry["carbohydrate"]

    return totals